[Norvig's setup code](https://github.com/norvig/pytudes/blob/master/ipynb/Advent%20of%20Code.ipynb)

In [1]:
#  Python 3.x
import operator as op

import re
import numpy as np
import math
import urllib.request

from collections import Counter, defaultdict, namedtuple, deque
from functools   import lru_cache
from itertools   import permutations, combinations, chain, cycle, product, islice
from heapq       import heappop, heappush

def Input(day):
    "Open this day's input file."
    filename = './input{}.txt'.format(day)
    return open(filename)

def transpose(matrix): return zip(*matrix)

def first(iterable): return next(iter(iterable))

def nth(iterable, n, default=None):
    "Returns the nth item of iterable, or a default value"
    return next(islice(iterable, n, None), default)

cat = ''.join

Ø   = frozenset() # Empty set
inf = float('inf')
BIG = 10 ** 999

def grep(pattern, lines):
    "Print lines that match pattern."
    for line in lines:
        if re.search(pattern, line):
            print(line)

def groupby(iterable, key=lambda it: it):
    "Return a dic whose keys are key(it) and whose values are all the elements of iterable with that key."
    dic = defaultdict(list)
    for it in iterable:
        dic[key(it)].append(it)
    return dic

def powerset(iterable):
    "Yield all subsets of items."
    items = list(iterable)
    for r in range(len(items)+1):
        for c in combinations(items, r):
            yield c

# 2-D points implemented using (x, y) tuples
def X(point): return point[0]
def Y(point): return point[1]

def neighbors4(point): 
    "The four neighbors (without diagonals)."
    x, y = point
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1))

def neighbors8(point): 
    "The eight neighbors (with diagonals)."
    x, y = point 
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1),
            (x+1, y+1), (x-1, y-1), (x+1, y-1), (x-1, y+1))

def cityblock_distance(p, q=(0, 0)): 
    "City block distance between two points."
    return abs(X(p) - X(q)) + abs(Y(p) - Y(q))

def euclidean_distance(p, q=(0, 0)): 
    "Euclidean (hypotenuse) distance between two points."
    return math.hypot(X(p) - X(q), Y(p) - Y(q))

def trace1(f):
    "Print a trace of the input and output of a function on one line."
    def traced_f(*args):
        result = f(*args)
        print('{}({}) = {}'.format(f.__name__, ', '.join(map(str, args)), result))
        return result
    return traced_f

def astar_search(start, h_func, moves_func):
    "Find a shortest sequence of states from start to a goal state (a state s with h_func(s) == 0)."
    frontier  = [(h_func(start), start)] # A priority queue, ordered by path length, f = g + h
    previous  = {start: None}  # start state has no previous state; other states will
    path_cost = {start: 0}     # The cost of the best path to a state.
    while frontier:
        (f, s) = heappop(frontier)
        if h_func(s) == 0:
            return Path(previous, s)
        for s2 in moves_func(s):
            new_cost = path_cost[s] + 1
            if s2 not in path_cost or new_cost < path_cost[s2]:
                heappush(frontier, (new_cost + h_func(s2), s2))
                path_cost[s2] = new_cost
                previous[s2] = s
    return dict(fail=True, front=len(frontier), prev=len(previous))
                
def Path(previous, s): 
    "Return a list of states that lead to state s, according to the previous dict."
    return ([] if (s is None) else Path(previous, previous[s]) + [s])

# [Day 1](https://adventofcode.com/2018/day/1)

For example, if the device displays frequency changes of +1, -2, +3, +1, then starting from a frequency of zero, the following changes would occur:

Current frequency  0, change of +1; resulting frequency  1.
Current frequency  1, change of -2; resulting frequency -1.
Current frequency -1, change of +3; resulting frequency  2.
Current frequency  2, change of +1; resulting frequency  3.
In this example, the resulting frequency is 3.

Here are other example situations:

+1, +1, +1 results in  3
+1, +1, -2 results in  0
-1, -2, -3 results in -6
Starting with a frequency of zero, what is the resulting frequency after all of the changes in frequency have been applied?


In [15]:
def day1_pt1(in_):
  return np.sum(list(map(int, in_)))

day1_pt1(Input(1))

590

## Part 2
Here are other examples:

* +1, -1 first reaches 0 twice.
* +3, +3, +4, -2, -4 first reaches 10 twice.
* -6, +3, +8, +5, -6 first reaches 5 twice.
* +7, +7, -2, -7, -4 first reaches 14 twice.

What is the first frequency your device reaches twice?

In [35]:
def day1_pt2(in_):
  xor_ = set()
  sum_ = 0
  for i in itertools.cycle(map(int, in_)):
    xor_.add(sum_)
    sum_ = sum_ + i
    if sum_ in xor_:
      return sum_

day1_pt2(list(Input(1)))

83445

# [Day 2](https://adventofcode.com/2018/day/2)

For example, if you see the following box IDs:

* abcdef contains no letters that appear exactly two or three times.
* bababc contains two a and three b, so it counts for both.
* abbcde contains two b, but no letter appears exactly three times.
* abcccd contains three c, but no letter appears exactly two times.
* aabcdd contains two a and two d, but it only counts once.
* abcdee contains two e.
* ababab contains three a and three b, but it only counts once.

Of these box IDs, four of them contain a letter which appears exactly twice, and three of them contain a letter which appears exactly three times. Multiplying these together produces a **checksum of 4 * 3 = 12.**

What is the checksum for your list of box IDs?

In [8]:
def day2_pt1(in_):
  def count23(s):
    counts = {v: k for k, v in Counter(s).items() if v in (2, 3)}
    return (
      counts.get(2, None) is not None, 
      counts.get(3, None) is not None)
  
  twos, threes = 0, 0
  for c2, c3 in map(count23, in_):
    twos += c2
    threes += c3
  return twos * threes

day2_pt1(Input(2))

6000

In [11]:
def diff_letters(s1, s2):
  diff = []
  for i, (c1, c2) in enumerate(zip(s1, s2)):
    if c1 != c2:
      diff.append(i)
    if len(diff) > 1:
      break
  return diff
  
def day2_pt2(in_):
  for i in range(len(in_)):
    for j in range(i + 1, len(in_)):
      diffs = diff_letters(in_[i], in_[j])
      if len(diffs) == 1:
        diff = diffs[0]
        return in_[i][:diff] + in_[i][diff+1:]

day2_pt2(list(Input(2)))        

'pbykrmjmizwhxlqnasfgtycdv\n'

# [Day 3](https://adventofcode.com/2018/day/3)


In [26]:
def parse_coordinates(line):
  # example: "#1 @ 850,301: 23x12"
  line = line.strip()
  
  at = line.index("@")
  colon = line.index(':')
  
  x, y = map(int, line[at+1:colon].split(','))
  w, h = map(int, line[colon+1:].split('x'))
  return int(line[1:at]), (x, y), (w, h)
  
def day3_pt1(in_):
  grid = np.zeros((1000, 1000), dtype=np.int32)
  for i, (x, y), (w, h) in map(parse_coordinates, in_):
    grid[x:x+w, y:y+h] += 1
  return np.count_nonzero(grid > 1)

day3_pt1(Input(3))

113576

In [36]:
def day3_pt2(in_):
  grid = np.zeros((1000, 1000), dtype=np.int32)
  non_overlap = set()
  for id_, (x, y), (w, h) in map(parse_coordinates, in_):
    uniques = np.unique(grid[x:x+w, y:y+h])
    if len(uniques) == 1 and uniques[0] == 0:
      non_overlap.add(id_)
    grid[x:x+w, y:y+h] = id_
    non_overlap.difference_update(uniques)
      
  return non_overlap

day3_pt2(Input(3))

{825}

# [Day 4](https://adventofcode.com/2018/day/4)

Using the format:
```
[1518-11-01 00:00] Guard #10 begins shift
[1518-11-01 00:05] falls asleep
[1518-11-01 00:25] wakes up
[1518-11-01 00:30] falls asleep
[1518-11-01 00:55] wakes up
[1518-11-01 23:58] Guard #99 begins shift
[1518-11-02 00:40] falls asleep
[1518-11-02 00:50] wakes up
[1518-11-03 00:05] Guard #10 begins shift
[1518-11-03 00:24] falls asleep
[1518-11-03 00:29] wakes up
[1518-11-04 00:02] Guard #99 begins shift
[1518-11-04 00:36] falls asleep
[1518-11-04 00:46] wakes up
[1518-11-05 00:03] Guard #99 begins shift
[1518-11-05 00:45] falls asleep
[1518-11-05 00:55] wakes up
```

Timestamps are written using `year-month-day hour:minute` format. The guard falling asleep or waking up is always the one whose shift most recently started. Because all asleep/awake times are during the midnight hour (00:00 - 00:59), **only the minute portion (00 - 59) is relevant** for those events.

While this example listed the entries in chronological order, **your entries are in the order you found them. You'll need to organize them before they can be analyzed.**

Strategy 1: Find the guard that has the most minutes asleep. What minute does that guard spend asleep the most?

What is the ID of the guard you chose multiplied by the minute you chose? (In the above example, the answer would be 10 * 24 = 240.)

In [3]:
in4 = list(Input(4))

Note, the sorting could technically have been done without extraction, as date format is ISO8601.

In [4]:
Event = namedtuple('Event', 'year month day hour minute action guard')
SWITCH, SLEEP, WAKE = -1, -2, -3


def structure(line):
  regex = r'\[(\d+)\-(\d+)\-(\d+) (\d+):(\d+)\][^#]*#?(\d*).*'
  year, month, day, hour, minute, guard = map(
    lambda x: int(x) if x else None, 
    re.match(regex, line).groups())
  action = SWITCH if guard else SLEEP if 'sleep' in line else WAKE
  return Event(year, month, day, hour, minute, action, guard)


def day4_pt1(in_):
  guards = defaultdict(lambda: np.array([0] * 60))
  asleep_min = defaultdict(int)
  on_duty = None
  for event in sorted(map(structure, in_)):
    if event.action == SWITCH:
      on_duty = event.guard
    elif event.action == SLEEP:
      sleep_minute = event.minute
    else:
      guards[on_duty][sleep_minute:event.minute] += 1
      asleep_min[on_duty] += (event.minute - sleep_minute)
  
  longest_asleep_id, _ = max(asleep_min.items(), key=lambda x: x[1])
  return longest_asleep_id * np.argmax(guards[longest_asleep_id])

day4_pt1(in4)

94040

In [5]:
def day4_pt2(in_):
  guards = defaultdict(lambda: np.array([0] * 60))
  on_duty = None
  for event in sorted(map(structure, in_)):
    if event.action == SWITCH:
      on_duty = event.guard
    elif event.action == SLEEP:
      sleep_minute = event.minute
    else:
      guards[on_duty][sleep_minute:event.minute] += 1
  
  longest_asleep_id, _ = max(guards.items(), key=lambda x: max(x[1]))
  return longest_asleep_id * np.argmax(guards[longest_asleep_id])

day4_pt2(in4)

39940

# [Day5](https://adventofcode.com/2018/day/5)

### For example:
- In aA, a and A react, leaving nothing behind.
- In abBA, bB destroys itself, leaving aA. As above, this then destroys itself, leaving nothing.
- In abAB, no two adjacent units are of the same type, and so nothing happens.
- In aabAAB, even though aa and AA are of the same type, their polarities match, and so nothing happens.

Now, consider a larger example, dabAcCaCBAcCcaDA:

```
dabAcCaCBAcCcaDA  The first 'cC' is removed.
dabAaCBAcCcaDA    This creates 'Aa', which is removed.
dabCBAcCcaDA      Either 'cC' or 'Cc' are removed (the result is the same).
dabCBAcaDA        No further actions can be taken.
After all possible reactions, the resulting polymer contains 10 units.
```

How many units remain after fully reacting the polymer you scanned? 

In [3]:
in5 = Input(5).read().strip()

In [12]:
def day5_pt1(s):
  stack = ['']
  for c in s:
    same_letter = stack[-1].lower() == c.lower()
    opposite_polarities = stack[-1] != c
    if same_letter and opposite_polarities:
      stack.pop()
    else:
      stack.append(c)
      
  return len(stack) - 1

day5_pt1(in5)

11590

## Part 2

One of the unit types is causing problems; it's preventing the polymer from collapsing as much as it should. Your goal is to figure out which unit type is causing the most problems, remove all instances of it (regardless of polarity), fully react the remaining polymer, and measure its length.

For example, again using the polymer dabAcCaCBAcCcaDA from above:

- Removing all A/a units produces dbcCCBcCcD. Fully reacting this polymer produces dbCBcD, which has length 6.
- Removing all B/b units produces daAcCaCAcCcaDA. Fully reacting this polymer produces daCAcaDA, which has length 8.
- Removing all C/c units produces dabAaBAaDA. Fully reacting this polymer produces daDA, which has length 4.
- Removing all D/d units produces abAcCaCBAcCcaA. Fully reacting this polymer produces abCBAc, which has length 6.

In this example, removing all C/c units was best, producing the answer 4.

What is the length of the shortest polymer you can produce by removing all units of exactly one type and fully reacting the result?

In [34]:
def day5_pt2(s):
  all_chars = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
  replace_char = lambda c: s.replace(c, '').replace(c.lower(), '')
  lengths = map(day5_pt1, map(replace_char, all_chars))
  return min(zip(lengths, all_chars))

day5_pt2(in5)

(4504, 'N')

# [Day 6](https://adventofcode.com/2018/day/6)

```
1, 1
1, 6
8, 3
3, 4
5, 5
8, 9
```

If we name these coordinates A through F, we can draw them on a grid, putting 0,0 at the top left.
Using the Manhattan distance, each location's closest coordinate can be determined, shown here in lowercase:

```
aaaaa.cccc
aAaaa.cccc
aaaddecccc
aadddeccCc
..dDdeeccc
bb.deEeecc
bBb.eeee..
bbb.eeefff
bbb.eeffff
bbb.ffffFf
```

Locations shown as . are equally far from two or more coordinates, and so they don't count as being closest to any.


In this example, the areas of coordinates A, B, C, and F are infinite - while not shown here, their areas extend forever outside the visible grid. However, the areas of coordinates D and E are finite: D is closest to 9 locations, and E is closest to 17 (both including the coordinate's location itself). Therefore, in this example, the size of the largest area is 17.

In [2]:
in6 = Input(6)

In [57]:
import itertools as it
Pt = namedtuple('Pt', 'x y id_', defaults=(None,)*3)


def manhattan(pt1, pt2):
  return abs(pt1.x - pt2.x) + abs(pt1.y - pt2.y)


def min_manhattan(pt, coords):
  manhattans = np.array([manhattan(pt, c) for c in coords])
  closest_i = np.where(manhattans == manhattans.min())[0]
  return coords[closest_i[0]].id_ \
    if len(closest_i) == 1 \
    else -1


def day6_pt1(in_):
  f_xy = lambda l: map(int, l.split(','))
  coords = [Pt(*f_xy(l), id_) for l, id_ in zip(in_, range(500))]
  
  xs = sorted(coords, key=lambda pt: pt.x)
  ys = sorted(coords, key=lambda pt: pt.y)
  
  limits = (Pt(xs[0].x, ys[0].y), Pt(xs[-1].x, ys[-1].y))
  grid = np.zeros((limits[1].y + 1, limits[1].x + 1))
  
  rx = range(limits[0].x, limits[1].x + 1)
  ry = range(limits[0].y, limits[1].y + 1)
  for (x, y) in it.product(rx, ry):
    grid[y, x] = min_manhattan(Pt(x, y), coords)
  
  grid = grid[limits[0].y:limits[1].y, limits[0].x:limits[1].x]  
  disallowed = set()
  for i in (0, -1):
    disallowed = disallowed.union(set(grid[i, :])).union(grid[:, i])
  
  return it.dropwhile(
    lambda kv: kv[0] in disallowed, 
    Counter(grid.reshape((-1,))).most_common())

next(day6_pt1(Input(6)))

(7.0, 3909)

## Part 2

For example, suppose you want the sum of the Manhattan distance to all of the coordinates to be less than 32. For each location, add up the distances to all of the given coordinates; if the total of those distances is less than 32, that location is within the desired region. Using the same coordinates as above, the resulting region looks like this:

```
..........
.A........
..........
...###..C.
..#D###...
..###E#...
.B.###....
..........
..........
........F.
```


What is the size of **the region** containing all locations which have a total distance to all given coordinates of less than 10000?

In [67]:
def sum_manhattan(pt, coords):
  return np.sum([manhattan(pt, c) for c in coords])
  

def day6_pt2(in_):
  f_xy = lambda l: map(int, l.split(','))
  coords = [Pt(*f_xy(l), id_) for l, id_ in zip(in_, range(500))]
  
  xs = sorted(coords, key=lambda pt: pt.x)
  ys = sorted(coords, key=lambda pt: pt.y)
  
  limits = (Pt(xs[0].x, ys[0].y), Pt(xs[-1].x, ys[-1].y))
  grid = np.zeros((limits[1].y + 1, limits[1].x + 1))
  
  rx = range(limits[0].x, limits[1].x + 1)
  ry = range(limits[0].y, limits[1].y + 1)
  sum_limit = 10000
 
  # note: there is only one such region
  return np.sum([
    sum_manhattan(Pt(x, y), coords) < sum_limit 
    for (x, y) in it.product(rx, ry)])
  
  
day6_pt2(Input(6))

36238

# [Day 7](https://adventofcode.com/2018/day/7)

```
Step C must be finished before step A can begin.
Step C must be finished before step F can begin.
Step A must be finished before step B can begin.
Step A must be finished before step D can begin.
Step B must be finished before step E can begin.
Step D must be finished before step E can begin.
Step F must be finished before step E can begin.
```

```
  -->A--->B--
 /    \      \
C      -->D----->E
 \           /
  ---->F-----
```

Your first goal is to determine the order in which the steps should be completed. If more than one step is ready, choose the step which is first alphabetically. In this example, the steps would be completed as follows:

- Only C is available, and so it is done first.
- Next, both A and F are available. A is first alphabetically, so it is done next.
- Then, even though F was available earlier, steps B and D are now also available, and B is the first alphabetically of the three.
- After that, only D and F are available. E is not available because only some of its prerequisites are complete. Therefore, D is completed next.
- F is the only choice, so it is done next.
- Finally, E is completed.

So, in this example, the correct order is CABDFE.

In [3]:
import heapq

def getdep(str_):
  return re.match(r'Step (.) .* step (.)', str_).groups()

def day7_pt1(in_):
  deps = defaultdict(list)
  invdeps = defaultdict(set)
  for b, a in map(getdep, in_):
    deps[b].append(a)
    invdeps[a].add(b)
  
  queue = list(set(deps.keys()).difference(set(invdeps.keys())))
  heapq.heapify(queue)
  while len(queue) > 0:
    top = heapq.heappop(queue)
    for child in deps.get(top, []):
      invdeps[child].remove(top)
      if len(invdeps[child]) == 0:
        heapq.heappush(queue, child)
      
    yield top
  

''.join(list(day7_pt1(Input(7))))

'BHMOTUFLCPQKWINZVRXAJDSYEG'

## Part 2

If multiple steps are available, workers should still begin them in alphabetical order.

Each step takes 60 seconds plus an amount corresponding to its letter: A=1, B=2, C=3, and so on. So, step A takes 60+1=61 seconds, while step Z takes 60+26=86 seconds. No time is required between steps.

To simplify things for the example, however, suppose you only have help from one Elf (a total of two workers) and that each step takes 60 fewer seconds (so that step A takes 1 second and step Z takes 26 seconds). Then, using the same instructions as above, this is how each second would be spent:



In [5]:
def task_duration(char_):
  return ord(char_) - ord('A') + 1 + 60

def day7_pt2(in_, n_elves=5):
  deps = defaultdict(list)
  invdeps = defaultdict(set)
  for b, a in map(getdep, in_):
    deps[b].append(a)
    invdeps[a].add(b)
  
  n_tasks = len(set(deps.keys()).union(invdeps.keys()))
  done_tasks = set()
  elves_free = set(range(n_elves))
  elves_working = dict()
  queue = list(set(deps.keys()).difference(set(invdeps.keys())))
  heapq.heapify(queue)
  
  for t in range((n_tasks) * task_duration('Z')):
    # earlier error was in putting `if len(done_tasks) ...` here
    for elf, (task, duration) in list(elves_working.items()):
      if duration == 1:
        elves_free.add(elf)
        done_tasks.add(task)
        del elves_working[elf]
      else:
        elves_working[elf] = (task, duration - 1)
    
    if len(done_tasks) == n_tasks:
      return t
    
    new_queue = []
    while len(elves_free) > 0 and len(queue) > 0:
      top = heapq.heappop(queue)
      if not len(invdeps.get(top, set()).difference(done_tasks)) == 0:
        heapq.heappush(new_queue, top)
        continue
        
      elves_working[elves_free.pop()] = (top, task_duration(top))
      
      for child in deps.get(top, []):
        if child not in queue and child not in new_queue:
          heapq.heappush(new_queue, child)
    queue = list(heapq.merge(queue, new_queue))


day7_pt2(Input(7))

877

# Day 8
The tree is made up of nodes; a single, outermost node forms the tree's root, and it contains all other nodes in the tree (or contains nodes that contain nodes, and so on).

Specifically, a node consists of:

A header, which is always exactly two numbers:
The quantity of child nodes.
The quantity of metadata entries.
Zero or more child nodes (as specified in the header).
One or more metadata entries (as specified in the header).
Each child node is itself a node that has its own header, child nodes, and metadata. For example:

```
2 3 0 3 10 11 12 1 1 0 1 99 2 1 1 2
A----------------------------------
    B----------- C-----------
                     D-----
```

In this example, each node of the tree is also marked with an underline starting with a letter for easier identification. In it, there are four nodes:

- A, which has 2 child nodes (B, C) and 3 metadata entries (1, 1, 2).
- B, which has 0 child nodes and 3 metadata entries (10, 11, 12).
- C, which has 1 child node (D) and 1 metadata entry (2).
- D, which has 0 child nodes and 1 metadata entry (99).

The first check done on the license file is to simply add up all of the metadata entries. In this example, that sum is `1+1+2+10+11+12+2+99=138`.

What is the sum of all metadata entries?

In [8]:
in8 = [int(x) for x in Input(8).read().split()]

In [60]:
def day8_pt1(in_):
  i = 0
  nchilds, nmetas, metas = [], [], []
  
  while i < len(in_):
    nchilds.append(in_[i])
    nmetas.append(in_[i + 1])
    i = i + 2
    
    while nchilds[-1] == 0:
      nmeta = nmetas.pop()
      metas.extend(in_[i:i + nmeta])
      i = i + nmeta
      
      nchilds.pop()
      if len(nchilds) == 0:
        break

      nchilds[-1] = nchilds[-1] - 1
  
  return np.sum(np.sum(metas))

day8_pt1(in8)

46829

## Part 2

In [92]:
class Node:
  def __init__(self):
    self.children = []
    self.metadata = []
    
  def get(self, nchild):
    return self.children[nchild-1]\
      if 0 < nchild <= len(self.children)\
      else None
  
  def value(self):
    if len(self.children) == 0:
      return np.sum(self.metadata)
    return np.sum([
      n.value() for n in [self.get(i) for i in self.metadata]
      if n is not None])

  
def day8_pt2(in_):  
  nchilds, nmetas, metas = [], [], []
  nodes = [Node()]
  i = 0
  
  while i < len(in_):
    node = Node(id)
    nodes.append(node)
    nchilds.append(in_[i])
    nmetas.append(in_[i + 1])
    i = i + 2
    
    while nchilds[-1] == 0:
      nmeta = nmetas.pop()
      node = nodes.pop()
      
      node.metadata.extend(in_[i:i + nmeta])
      i = i + nmeta
      nodes[-1].children.append(node)
      
      nchilds.pop()
      if len(nchilds) == 0:
        break

      nchilds[-1] = nchilds[-1] - 1
      
  guard = nodes.pop()
  return guard.children[0].value()

day8_pt2(in8)

37450

## Day9

Elves are playing a game placing marbles:

First, the marble numbered 0 is placed in the circle. At this point, while it contains only a single marble, it is still a circle: the marble is both clockwise from itself and counter-clockwise from itself. This marble is designated the current marble.

Then, each Elf takes a turn placing the lowest-numbered remaining marble into the circle between the marbles that are 1 and 2 marbles clockwise of the current marble. (When the circle is large enough, this means that there is one marble between the marble that was just placed and the current marble.) The marble that was just placed then becomes the current marble.

However, if the marble that is about to be placed has a number which is a multiple of 23, something entirely different happens. First, the current player keeps the marble they would have placed, adding it to their score. In addition, the marble 7 marbles counter-clockwise from the current marble is removed from the circle and also added to the current player's score. The marble located immediately clockwise of the marble that was removed becomes the new current marble.

You have to determine what is the winning player's score.


In [117]:
def day9_pt1(nplayers=416, last=71617):
  scores = np.zeros((nplayers,))
  marbles = [0, 2, 1]
  current_index = 1
  player = 0
  for marble in range(3, last + 1):    
    if marble % 23 == 0:
      scores[player] += marble
      current_index -= 7
      scores[player] += marbles.pop(current_index)
    else:
      current_index = (current_index + 2) % len(marbles)
      marbles.insert(current_index, marble)
      
    player = (player + 1) % len(scores)
  return np.max(scores)

day9_pt1()

436720.0

## Part 2
Larger value, use deque for efficient addition/removal.

In [158]:
def day9_pt2(nplayers=416, last=7161700):
  scores = defaultdict(int)
  marbles = deque([0])
  current_index = 1
  player = 0
  for marble in range(1, last + 1):
    if marble % 23 == 0:
      marbles.rotate(7)
      scores[player] += marble
      scores[player] += marbles.pop()
      marbles.rotate(-1)
    else:
      marbles.rotate(-1)
      marbles.append(marble)  
    player = (player + 1) % nplayers
    
  return np.max(list(scores.values()))

day9_pt2()

3527845091